In [1]:
# Force use of development local repository.
# The repository must be the russell_dev branch of https://github.com/russelljjarvis/neuronunit
# and the docker build is: docker-stacks/neuronunit-optimization
# There are more graceful ways of achieving this.
# The best way will be to integrate this doc chapter5 and its supporting code 
# from https://github.com/russelljjarvis/neuronunit -> https://github.com/scidash/neuronunit

# On my local machine the development repo of neuronunit is located here:
# $HOME/git
# I navigate to that directory and execute:

# docker run -p 8888:8888 -v \
# `pwd`:/home/jovyan/mnt scidash/neuronunit-optimization \
# jupyter notebook --ip=0.0.0.0 --NotebookApp.token=\"\" \ 
# --NotebookApp.disable_check_xsrf=True 

import sys
import os
THIS_DIR = os.path.dirname(os.path.realpath('chapter5.ipynb'))
this_nu = os.path.join(THIS_DIR,'../')
sys.path.insert(0,this_nu)
import neuronunit


In [ ]:

from neuronunit.tests import get_neab

from neuronunit.tests import utilities as outils                                    
from neuronunit.tests import model_parameters as modelp
import numpy as np
model = outils.model          
from sciunit import scores


Getting Rheobase cached data value for from AIBS dataset 354190013
attempting to recover from pickled file
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml
Mechanisms already loaded from path: /home/jovyan/mnt/neuronunit/tests/NeuroML2.  Aborting.
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml


In [ ]:

class Score:
    def __init__(self, score):
        self.score = score

class Test:
    def _optimize(self, model,modelp):
        '''
        The implementation of optimization, consisting of implementation details.
        Inputs a model, and model parameter ranges to expore
        Private method for programmer designer.
        Outputs the optimal model, its attributes and the low error it resulted in.
        '''
        from neuronunit.tests import nsga
        gap = nsga.GAparams(model)
        # The number of generations is 3
        gap.NGEN = 3
        # The population of genes is 12
        gap.MU = 12 
        gap.BOUND_LOW = [ np.min(i) for i in modelp.model_params.values() ]
        gap.BOUND_UP = [ np.max(i) for i in modelp.model_params.values() ]
        # call the actual Genetic Algorithm with Optimization parameters: number of generation (NGEN = 3)
        # and gene population size (MU = 12)
        vmpop, pop, invalid_ind, pf = nsga.main(gap.NGEN,gap.MU,model,modelp)
        attributes = [ i.attrs for i in vmpop ]
        rheobases = [ i.rheobase for i in vmpop ]
        scores = [ i.score for i in vmpop ]        
        parameters = [ i.attrs for i in vmpop ]
        
        data_tuples = (vmpop, parameters, scores, pop, invalid_ind, pf, rheobases)
        return pop, data_tuples
        
    def _get_optimization_parameters(self, data_tuples):
        # Your specific unpacking of tuples that _optimize returns
        # vmpop, parameters, scores, pop, invalid_ind, pf, rheobases)
        _ , parameters , scores, _ , _ , _ , _ = zip(*data_tuples)
        return parameters,scores

    def optimize(self, model, modelp):
        '''
        # The Class users version of optimize
        # where details are hidden in _optimizae
        
        # Inputs: 
        # a Izhihikitch model specified in NML, but implemented with a NEURONbackend type
        # from neuronunit. Modelp a formatated dictionary of model parameters
        # where keys are Izhikitich parameters, and values are parameter ranges.
        # Outputs:
        # the optimal model, the scores as pandas data frame.
        # data_tuples: other data about models from the converged gene population
        # like resulting rheobase values from the converged genes, pandas score arrays
        # the genes corresponding to attributes of the pareto front (in a raw format).
        '''

        # Do optimization including repeated calls to judge
        models, data_tuples = self._optimize(model,modelp)
        parameters, scores = self._get_optimization_parameters(data_tuples)
        # this a way of looking at solved model parameters, ie candidate solutions from 
        # the pareto front.
        # scores is a list of pandas dataframes for the converged gene population.
        # It might be good to convert it into one big panda table if I knew how.
        return model, scores, data_tuples

    
t = Test()
model,scores,data_tuples = t.optimize(model,modelp)   

<module 'neuronunit.tests.model_parameters' from '/home/jovyan/mnt/docs/../neuronunit/tests/model_parameters.py'>
0 -57.6008139654
1 -30.1313130423
2 0.119270369187
3 -3.50310135304e-09
4 -55.1384937114
5 35.7506709947
6 0.000757279791904
7 0.000101447133962
8 0.132159334129
9 -63.6285331396
0 -63.8641875221
1 -36.2883936303
2 0.666379202859
3 -4.8282623178e-09
4 -57.6748005892
5 38.9964389513
6 0.00128296738157
7 9.50168028557e-05
8 0.0527165694785
9 -63.9441474743
0 -64.6140168849
1 -32.2208488945
2 0.822547708396
3 -3.52924195923e-09
4 -56.0202529846
5 31.1189610363
6 0.000732282738459
7 0.000106300376041
8 0.184822544895
9 -67.0297376846
0 -52.5038526973
1 -33.1947672995
2 0.742546636473
3 -4.59816809666e-09
4 -57.2654090368
5 39.3677780472
6 0.000996733121754
7 9.92884782329e-05
8 0.126760136313
9 -68.8887747416
0 -67.6634127351
1 -37.5734976436
2 0.23287860647
3 -3.86963851404e-09
4 -58.55392644
5 37.4584283464
6 0.00131608275648
7 0.000104946873814
8 0.09270359691
9 -60.04406596

/opt/conda/lib/python3.5/site-packages/scoop-0.7.2.0-py3.5.egg/scoop/fallbacks.py:46: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().


 0.000902086771999
7 0.000105562620638
8 0.0951299265074
9 -65.7042507092
0 -45.9933954038
1 -30.8308245276
2 0.36854394664
3 -3.80360182871e-09
4 -56.2960421179
5 39.0781792068
6 0.000819576096568
7 9.02054661353e-05
8 0.110102403678
9 -51.2992775388
0 -72.7780779396
1 -34.1960333945
2 0.656960767758
3 -4.11665133239e-09
4 -57.5435347142
5 33.1744758873
6 0.000730651008041
7 9.58953192623e-05
8 0.070788343779
9 -59.9173751965
0 -63.7816357857
1 -45.012221586
2 0.477914019413
3 -3.55391908085e-09
4 -55.5142565139
5 32.8090109689
6 0.00118317810079
7 9.68184274825e-05
8 0.164391755066
9 -72.7715594976
0 -60.5638159601
1 -41.5063191083
2 0.599319469522
3 -4.74175102426e-09
4 -57.641119681
5 39.8562828668
6 0.000746248588335
7 0.000101430038034
8 0.100698882434
9 -60.8433056353
0 -63.0612530862
1 -43.1824131201
2 0.544064531297
3 -3.81484149762e-09
4 -55.1854674657
5 36.1736517795
6 0.00136493165653
7 0.000102753780264
8 0.0574174050674
9 -68.3862879159
0 -65.169686728
1 -42.8666726643
2 

In [ ]:
modelp
import pandas as pd
models = data_tuples[1]
sc = pd.DataFrame(scores[0])
for j,i in enumerate(models):
        i.name = attributes[j]
sc

In [ ]:
data = [ models[0].name ]
model_values0 = pd.DataFrame(data)        
model_values0

In [ ]:
rheobases=data_tuples[5][0]

In [ ]:
data = [ models[0].name ]
model_values0 = pd.DataFrame(data)        
model_values0


In [ ]:
rheobases[0]


In [ ]:

sc1 = pd.DataFrame(scores[1])
sc1


In [ ]:
rheobases[1]


In [ ]:
data=[ models[1].name ]
model_values1 = pd.DataFrame(data)        
model_values1

In [ ]:
models[1].name        

In [ ]:

import pickle
import pandas as pd

# I have knowingly violated Github conventions by adding data (a pickled file, as well as sources to the repository). 
# The justification being that ground_error (the ground truth to compare against Genetic Algorithm outputs). 
# Takes a prohibitively long time to generate, and therefore detracts from notebooking philosophy.

try:
    ground_error = pickle.load(open('big_model_evaulated.pickle','rb'))
except:
    # The exception code is only skeletal, it would actually work, but its the right principles.
    print('{0} it seems the error truth data does not yet exist, lets create it now '.format(str(False)))
    ground_error = list(futures.map(outils.func2map, ground_truth))
    pickle.dump(ground_error,open('big_model_evaulated.pickle','wb'))

# ground_error_nsga=list(zip(vmpop,pop,invalid_ind))
# pickle.dump(ground_error_nsga,open('nsga_evaulated.pickle','wb'))
# Get the differences between values obtained via brute force, and those obtained otherwise:
sum_errors = [ i[0] for i in ground_error ]
composite_errors = [ i[1] for i in ground_error ]
attrs = [ i[2] for i in ground_error ]
rheobase = [ i[3] for i in ground_error ]

indexs = [i for i,j in enumerate(sum_errors) if j==np.min(sum_errors) ][0]
indexc = [i for i,j in enumerate(composite_errors) if j==np.min(composite_errors) ][0]
#assert indexs == indexc
vmpop = data_tuples[0]
df_0 = pd.DataFrame([ (k,v,vmpop[0].attrs[k],float(v)-float(vmpop[0].attrs[k])) for k,v in ground_error[indexc][2].items() ])
df_1 = pd.DataFrame([ (k,v,vmpop[1].attrs[k],float(v)-float(vmpop[1].attrs[k])) for k,v in ground_error[indexc][2].items() ])




In [ ]:
#These are the differences in attributes found via brute force versus the genetic algorithm. For the top two candidates.

df_0

In [ ]:
df_1
